# Backend file storage algorithm

## Objective
The system will automatically put the files in different folders (on cloud or on your local computer) based on the real time location (name of the city) of the devices, such as “Vancouver”, “Surrey”, “Victoria”, “Burnaby”, “Coquitlam”, etc. If a new location is used, the system will automatically create a new folder for this new location and find out the city name of the location to use as the folder name, if the system cannot find the name of the city, the system will ask the user. The system will confirm after uploading (success/fail).

## Developing version

### Random location generation based on Postcode
Generating a random Canadian postal code involves creating a string that matches the format of Canadian postal codes: A1A 1A1, where "A" represents an alphabet character and "1" represents a digit.

In [1]:
import random
import string

def generate_random_postal_code():
    # Characters allowed in different positions of the postal code
    alphabet = string.ascii_uppercase
    digits = string.digits

    # Generate each part of the postal code
    first_alpha = random.choice(alphabet)
    first_digit = random.choice(digits)
    second_alpha = random.choice(alphabet)

    third_digit = random.choice(digits)
    fourth_alpha = random.choice(alphabet)
    fifth_digit = random.choice(digits)

    # Combine all parts into the standard format
    postal_code = f"{first_alpha}{first_digit}{second_alpha} {third_digit}{fourth_alpha}{fifth_digit}"

    return postal_code

# Generate and print a random Canadian postal code
print(generate_random_postal_code())

P0O 9Y4


### address searcher based on postcode

In [64]:
from geopy.geocoders import Nominatim

def get_address(postal_code):
    geolocator = Nominatim(user_agent="location_based_smart_drive")
    location = geolocator.geocode(postal_code, country_codes="CA")
    if location:
        print(f'Address: {location}')
        return location.address
    else:
        print(f'No address found for postal code: {postal_code}')
        return None

In [15]:
# Example usage:
postal_code = "V9B 6A2"  # Replace with the postal code you are looking up
address = get_address(postal_code)
if address:
    print(f'Address: {address}')
else:
    print(f'No address found for postal code: {postal_code}')

Address: Langford, Capital Regional District, V9B 6A2, British Columbia, Canada


In [19]:
costco = "V9B 6A2"
downtown = "V8W 1H2"
UVic = "V8P 5C2"
sidney = "V8L 1X5"

In [21]:
postal_code = sidney
address = get_address(postal_code)
if address:
    print(f'Address: {address}')
else:
    print(f'No address found for postal code: {postal_code}')

Address: Sidney, Capital Regional District, V8L 1X5, British Columbia, Canada


In [32]:
postal_code = downtown
address = get_address(postal_code)
if address:
    print(f'Address: {address}')
else:
    print(f'No address found for postal code: {postal_code}')

Address: Downtown, Victoria, Capital Regional District, V8W 1H2, British Columbia, Canada


Split the long address into 4 levels

In [37]:
add_lst = address.split(", ")
print(add_lst)
add_lst.reverse()
add_level = add_lst[1:2] + add_lst[3:5]
add_level = [x.replace(" ","_") for x in add_level]
print(add_level)

['Downtown', 'Victoria', 'Capital Regional District', 'V8W 1H2', 'British Columbia', 'Canada']
['British_Columbia', 'Capital_Regional_District', 'Victoria']


### get the directory

In [40]:
# Use the %pwd magic command to get the current working directory
current_directory = %pwd
print(current_directory)
# Now to get the father (parent) directory:
import os
father_directory = os.path.dirname(current_directory)
print(father_directory)
test_repository = os.path.join(father_directory, "Backend test")
print(test_repository)
print(os.path.exists(test_repository))

/Users/wanrylin/Python code/ECE 569 IOT/Backend
/Users/wanrylin/Python code/ECE 569 IOT
/Users/wanrylin/Python code/ECE 569 IOT/Backend test
False


Create a new folder for testing file storage algorithm

In [49]:
if not os.path.exists(test_repository):
    os.mkdir(test_repository)
    print(f'Directory {test_repository} created.')
else:
    print(f'Directory {test_repository} already exists.')
# create a folder to store all the generated files
test_file_folder = os.path.join(test_repository, "test file")
if not os.path.exists(test_file_folder):
    os.mkdir(test_file_folder)
    print(f'Directory {test_file_folder} created.')
else:
    print(f'Directory {test_file_folder} already exists.')

Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/test file created.


### location based folder process
find if the location exists in current directory. Exist, return ture flag. Not exist, reuturn false flag and create new folders

In [46]:
def ensure_directories_exist(directory_list, father_directory):
    current_directory = father_directory
    flag = "True"
    for directory in directory_list:
        # Construct the path to the next directory
        current_directory = os.path.join(current_directory, directory)
        # Check if the directory exists, if not create it
        if not os.path.exists(current_directory):
            flag = "False"
            os.mkdir(current_directory)
            print(f'Directory {current_directory} created.')
        else:
            print(f'Directory {current_directory} already exists.')
    # return the father path of the file, flag of path existance(T:exist, F:not)
    return current_directory, flag

# Call the function
file_father_path, exist_flag = ensure_directories_exist(add_level, test_repository)
print(file_father_path, exist_flag)

Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria already exists.
/Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria False


### move file

generate file

In [55]:
# create a file for test
file_name = postal_code + ".txt"
# content is the address
file_content = address
# file path
source_file_path = os.path.join(test_file_folder, file_name)
# Open the file with write ('w') permission. If the file doesn't exist, Python will create it.
with open(source_file_path, 'w') as file:
    # Write the content to the file
    file.write(file_content)

Copy file to the location folder

In [56]:
import shutil

# Specify the destination path
destination_path = os.path.join(file_father_path,file_name)

# Use shutil.copy2() to copy the file
shutil.copy2(source_file_path, destination_path)


'/Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria/V8W 1H2.txt'

In [77]:
import shutil

def copy_file(source_file_path, file_father_path,file_name):
    # Specify the destination path
    destination_path = os.path.join(file_father_path,file_name)
    try:
        # Attempt to copy the file
        dest = shutil.copy2(source_file_path, destination_path)
        return f'Copy successful: {dest}'
    except FileNotFoundError as e:
        return f'File not found: {e}'
    except PermissionError as e:
        return f'Permission error: {e}'
    except Exception as e:
        # Catch any other exceptions
        return f'An error occurred: {e}'



# Call the function and print the result
result = copy_file(source_file_path, file_father_path,file_name)
print(result)

Copy successful: /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria/V8W 1H2.txt


# Intergrate all the functions

In [60]:
# find if the directory exist or not and create the folder if not exist
def ensure_path_exist(directory):
    flag = "True"
    # Check if the directory exists, if not create it
    if not os.path.exists(directory):
        flag = "False"
        os.mkdir(directory)
        print(f'Directory {directory} created.')
    else:
        print(f'Directory {directory} already exists.')
    # return the father path of the file, flag of path existance(T:exist, F:not)
    return directory, flag

In [83]:
# Get the directory according to the location
def get_path(address):
    # turn address str into str list
    add_lst = address.split(", ")
    # in country, province, postal code, city, town order
    add_lst.reverse()
    # just use province, city and town
    add_level = add_lst[1:2] + add_lst[3:5]
    # replace space by _
    add_level = [x.replace(" ", "_") for x in add_level]
    # check whether "Backend test" folder exist or not
    # Get the current working directory
    current_directory = os.getcwd()
    # Now to get the father (parent) directory:
    father_directory = os.path.dirname(current_directory)
    test_repository = os.path.join(father_directory, "Backend test")
    # find it "test_repository" exist or not
    test_repository,test_repository_flag = ensure_path_exist(test_repository)
    # create a folder to store all the generated files
    test_file_folder = os.path.join(test_repository, "test file")
    # find it "test_file_folder" exist or not
    test_file_folder,test_file_folder_flag = ensure_path_exist(test_file_folder)
    flag = "True"
    add_path = test_repository
    for add_directory in add_level:
        # Construct the path to the next directory
        add_path = os.path.join(add_path, add_directory)
        # Check if the directory exists, if not create it
        add_path,add_flag = ensure_path_exist(add_path)

    return add_path,add_flag,test_repository_flag,test_file_folder

In [78]:
# create test file and store in test file folder
def create_test_file(postal_code,address,test_file_folder):
    # create a file for test
    file_name = postal_code + ".txt"
    # content is the address
    file_content = address
    # file path
    source_file_path = os.path.join(test_file_folder, file_name)
    # Open the file with write ('w') permission. If the file doesn't exist, Python will create it.
    with open(source_file_path, 'w') as file:
        # Write the content to the file
        file.write(file_content)

    return source_file_path,file_name

In [86]:
# get test postal code
def get_postal_code(mode):
    # mode 1: randomly generating
    if mode == 0:
        loop = 0
        while True:
            random_postal_code = generate_random_postal_code()
            loop = loop + 1
            address = get_address(random_postal_code)
            if address:
                print(f"iteration times: {loop}")
                break
    elif mode == 1:
        costco = "V9B 6A2"
        downtown = "V8W 1H2"
        UVic = "V8P 5C2"
        sidney = "V8L 1X5"
        random_postal_code = downtown

    print(f"Postal code: {random_postal_code}")
    return random_postal_code

In [87]:
# get test postal code
postal_code = get_postal_code(1)
# get address according to the postal code
address = get_address(postal_code)
# get file path according to the address
file_path,add_flag,test_repository_flag,test_file_folder_flag = get_path(address)
# create test file
source_file_path,file_name = create_test_file(postal_code,address,test_file_folder)
# move test file
result = copy_file(source_file_path, file_path,file_name)
print(result)

Postal code: V8W 1H2
Address: Downtown, Victoria, Capital Regional District, V8W 1H2, British Columbia, Canada
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/test file already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District already exists.
Directory /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria created.
Copy successful: /Users/wanrylin/Python code/ECE 569 IOT/Backend test/British_Columbia/Capital_Regional_District/Victoria/V8W 1H2.txt
